# Parse website

In [ ]:
from bs4 import BeautifulSoup
import requests
url = "https://kworb.net/spotify/country/global_weekly_totals.html"
response = requests.get(url)
response.text[:100] # Access the HTML with the text property
#print(response.text)

In [ ]:
# Some code from here https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/
import pandas as pd
from bs4 import BeautifulSoup
class HTMLTableParser:

    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table,self.parse_html_table(table))\
                for table in soup.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):

            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)

            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                if column_marker == 1:
                    df.iat[row_marker,column_marker] = column.get_text() + "||" + column.find_all('a')[1]['href']
                else:
                    df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1

        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass

        return df


In [ ]:
hp = HTMLTableParser()
table = hp.parse_url("https://kworb.net/spotify/country/global_weekly_totals.html")[0][1]

In [ ]:
table['Artist and Title'][0].split("||")[1].split('/')[2].split('.')[0]

In [ ]:
table.to_csv("new_data.csv", encoding="utf-8", header=True, index=False)

# Integrate with Spotify API

In [ ]:
from __future__ import print_function    # (at top of module)
import warnings
warnings.filterwarnings('always')
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import sys
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import seaborn as sns
import config


# Spotify API Setup
client_credentials_manager = SpotifyClientCredentials(config.client_id, config.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Enables verbose JSON requests tracing
sp.trace=False

In [ ]:
# File name to write to
fileName = "newdata_combined_2013_2019.csv"

# Columns for my pandas DataFrame in which we will keep the data
columns = ["song_id","song_title", "artist", "popularity", "total_no_streams", "energy", "liveness", "tempo"
          , "speechiness", "acousticness", "instrumentalness", "time_signature", "danceability",
          "key", "duration", "loudness", "valence", "mode"]

# Actual data structure for the data
myData = []

count = 0
for index, row in table.iterrows():
    track = sp.track("spotify:track:" + row['Artist and Title'].split("||")[1].split('/')[2].split('.')[0])
    #print(result['tracks']['items'][0]['name'])

    trackId = track['uri']
    songTitle = track['name']
    popularity = int(track['popularity'])

    # A song might have more than one artist so we make a list of all of them
    artistName = []
    for artist in track['artists']:
        artistName.append(artist['name'])

    # Get features for the track
    features = sp.audio_features([trackId])

    # If the feature array is empty this usually means something has gone wrong 
    # with the request so this stops the program from failing in that case
    if features[0] != None :
        energy = features[0]['energy']
        liveness = features[0]['liveness'] 
        tempo = features[0]['tempo']
        speechiness = features[0]['speechiness']
        acousticness = features[0]['acousticness']
        instrumentalness = features[0]['instrumentalness']
        time_signature = features[0]['time_signature']
        danceability = features[0]['danceability']
        key = features[0]['key']
        duration_ms = features[0]['duration_ms']
        loudness = features[0]['loudness']
        valence = features[0]['valence']
        mode = features[0]['mode']
        total_no_streams = int(row['Total'].replace(',', ''))
    newRow = [trackId,songTitle, artistName, popularity, total_no_streams,energy, liveness, tempo, speechiness, acousticness, instrumentalness, time_signature,
              danceability, key, duration_ms, loudness, valence, mode]
    #print(newRow)
    # Add the new row to our existing data
    myData.append(newRow)
    count += 1
    if(count % 100 == 0):
        print("Processed " + str(count) + " songs so far.")
    time.sleep(0.1)
print("Finished processing.")
df = pd.DataFrame(myData, columns=columns)
df.head()
print("Writing file to CSV...")
df.to_csv(fileName, encoding="utf-8", header=True, index=False)
print("Done.")
df.head()

In [ ]:
# Read the data from the CSV to make sure everything is fine
data = pd.read_csv("google_cleaned_data.csv")
data.head()

In [ ]:
print("Number of entries in original data: " + str(len(data.index)))
data['popularity'] = pd.to_numeric(data['popularity'])